In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy.io import wavfile
import os
#import control.pzmap as pzm
#from control import TransferFunction

In [ ]:
cwd = os.getcwd() + '/'

## Auto-regressive (AR) models

In [ ]:
N = 10000
axis = np.arange(N)
ssq_e = 1

In [ ]:
def AR(alpha, sigma, num, is_init_vals=False, initial_values=None):
    # order of AR model
    P = alpha.shape[0]
    # observation vector
    data = np.zeros(num)
    # first P values must be defined for the model to work correctly
    # default to zero initial values
    if is_init_vals:
        data[:P] = initial_values
    else:
        data[:P] = np.zeros(P)
    # iterate over all data points following the initial values
    for i in range(P, num):
        term = 0
        # sum over all P coefficients
        for j in range(1, P+1):
            term += alpha[j-1]*data[i-j]
        # add innovation term
        term += sigma*np.random.normal(size=1)
        data[i] = term
    return data

In [ ]:
P = 150
coeffs = np.random.normal(0, 0.04, P)

In [ ]:
def plot_poles(coefficients, samps=1000):
    polynom = np.pad(-coefficients, (1, 0), mode='constant', constant_values=1)
    poles_f = np.roots(polynom).astype(np.complex128)
    fig, ax = plt.subplots()
    ax.set_box_aspect(aspect=1.)
    t = np.linspace(0, 2*np.pi, 1000)
    ax.plot(np.cos(t), np.sin(t))
    ax.scatter(poles_f.real, poles_f.imag, marker='X', linewidth=0.05)
    ax.scatter(0, 0, marker='o', linewidth=0.05)
    ax.grid(b=True, linewidth=0.1)
    plt.xlim(-2, 2)
    plt.ylim(-2, 2)
    plt.title('Largest pole: {}'.format(np.max(np.abs(poles_f))))
    return poles_f
poles = plot_poles(coeffs)
poles

In [ ]:
X = AR(coeffs, np.sqrt(ssq_e), N, is_init_vals=False, initial_values=np.random.normal(size=coeffs.shape[0]))

In [ ]:
plt.figure()
plt.plot(axis, X)
plt.title('AR {}. Coefficients: {}'.format(P, coeffs))
plt.xlabel('n')
plt.ylabel('Xn')

In [ ]:
plt.figure()
plt.plot(np.fft.fftshift(np.fft.fftfreq(N-P)), ((np.square(np.abs(np.fft.fftshift(np.fft.fft(X[P:])))))))
plt.xlabel('Normalised frequency')
plt.ylabel('Squared magnitude')
plt.title('Squared DFT components of AR filter')

## Maximum likelihood

In [ ]:
P_est = P

In [ ]:
def build_AR_matrix(data, p):
    # retrieve n for matrix dimension
    n = data.shape[0]
    # initialise G as (n-p)x(p) matrix
    mat = np.zeros((n-p,p))
    # build matrix column-wise, taking observations from data
    for i in range(p):
        mat[:,i] = data[p-i-1:n-i-1]
    return mat

In [ ]:
def get_alpha_ml(data, P_estimate):
    data_trunc = data[P_estimate:]
    G_mat = build_AR_matrix(data, P_estimate)
    return np.matmul(np.matmul(np.linalg.inv(np.matmul(G_mat.T, G_mat)), G_mat.T), data_trunc)

In [ ]:
alpha_ML = get_alpha_ml(X, P_est)

In [ ]:
X_ml = np.matmul(build_AR_matrix(X, P_est), alpha_ML)

In [ ]:
plt.figure()
plt.plot(np.arange(N), X, label='true')
plt.plot(np.arange(N)[P_est:], np.roll(X_ml, -1), label='estimated')
plt.legend()

In [ ]:
np.mean(np.square(X[P_est:]-X_ml))

## Bayesian

In [ ]:
m_a = 1000*np.ones(P_est)
ssq_a = 1
inv_Covar_a = np.linalg.inv(ssq_a * np.eye(P_est))
m_a_true = 0*np.ones(P_est)
ssq_a_true = 0
inv_Covar_a_true = np.linalg.inv(ssq_a * np.eye(P_est))

In [ ]:
def get_alpha_map(data, P_estimate, noise_variance, prior_mean, inv_prior_covar):
    data_trunc = data[P_estimate:]
    G_mat = build_AR_matrix(data, P_estimate)
    Phi = np.matmul(G_mat.T, G_mat) + noise_variance*inv_prior_covar
    Theta = np.matmul(G_mat.T, data_trunc) + noise_variance*np.matmul(inv_prior_covar, prior_mean)
    return np.matmul(np.linalg.inv(Phi), Theta)

In [ ]:
alpha_MAP = get_alpha_map(X, P_est, ssq_e, m_a, inv_Covar_a)
alpha_MAP_true = get_alpha_map(X, P_est, ssq_e, m_a_true, inv_Covar_a_true)

In [ ]:
X_map = np.matmul(build_AR_matrix(X, P_est), alpha_MAP)
X_map_true = np.matmul(build_AR_matrix(X, P_est), alpha_MAP_true)

In [ ]:
plt.figure()
plt.subplot(311)
plt.plot(np.arange(N), X)
plt.plot(np.arange(start=-1, stop=N-1)[P_est:], X_ml)
plt.title('Estimated processes. Blue: True, Orange: Estimated')
plt.ylabel('ML')
plt.subplot(312)
plt.plot(np.arange(N), X)
plt.plot(np.arange(start=-1, stop=N-1)[P_est:], X_map_true)
plt.ylabel('True MAP')
plt.subplot(313)
plt.plot(np.arange(N), X)
plt.plot(np.arange(start=-1, stop=N-1)[P_est:], X_map)
plt.xlabel('n')
plt.ylabel('Adjusted MAP')

In [ ]:
#np.mean(np.square(X[P_est:]-np.roll(X_ml, -1)))
np.mean(np.square(X[P_est:]-X_ml))

In [ ]:
#np.mean(np.square(X[P_est:]-np.roll(X_map, -1))
np.mean(np.square(X[P_est:]-X_map))

In [ ]:
#np.mean(np.square(X[P_est:]-np.roll(X_map_true, -1)))
np.mean(np.square(X[P_est:]-X_map_true))

In [ ]:
def get_log_model_evidence(data, noise_variance, prior_mean, inv_prior_covar, Gen):
    p_est = Gen.shape[1]
    data_trunc = data[p_est:]
    Num_samps = data_trunc.shape[0]
    phi = np.matmul(Gen.T, Gen) + noise_variance * inv_prior_covar
    theta = np.matmul(Gen.T, data_trunc) + noise_variance * np.matmul(inv_prior_covar, prior_mean)
    th_map = np.matmul(np.linalg.inv(phi), theta)
    sign_C, log_det_C_inv = np.linalg.slogdet(inv_prior_covar)
    sign_P, log_det_phi = np.linalg.slogdet(phi)
    
    t1 = -Num_samps*np.log(2*np.pi)/2
    t2 = sign_C*log_det_C_inv/2
    t3 = -sign_P*log_det_phi/2
    t4 = -(Num_samps-2)*np.log(noise_variance)/2
    
    t5 = np.sum(np.square(data_trunc))
    t6 = noise_variance * np.linalg.multi_dot((prior_mean, inv_prior_covar, prior_mean))
    t7 = np.matmul(theta.T, th_map)
    
    
    return t1+t2+t3+t4 - (t5+t6-t7)/(2*noise_variance)

In [ ]:
evidences = np.zeros(200)
for i in range(200):
    prior_m = 1*np.ones(i+1)
    prior_c_inv = np.linalg.inv(ssq_a * np.eye(i+1))
    gi = build_AR_matrix(X, i+1)
    evidences[i] = get_log_model_evidence(X, ssq_e, 100*prior_m, 1*prior_c_inv, gi)
np.argmax(evidences) + 1

In [ ]:
plt.figure()
plt.plot(np.arange(1, 200+1), evidences)
plt.xlabel('Model order')
plt.ylabel('Log model evidence')
plt.title('Evidences for true prior parameters')

In [ ]:
mean_sq_errors = np.zeros(300)
for i in range(300):
    gi = build_AR_matrix(X, i+1)
    #mean_sq_errors[i] = np.mean(np.square(X[i+1:] - np.roll(np.matmul(gi, get_alpha_ml(X, i+1)), -1)))
    mean_sq_errors[i] = np.mean(np.square(X[i+1:] - np.matmul(gi, get_alpha_ml(X, i+1))))

plt.figure()
plt.plot(np.arange(300), mean_sq_errors)
plt.xlabel('Model order')
plt.ylabel('Mean squared residual error')
plt.title('True model order: {}'.format(P))

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot(np.arange(X[P_est:].shape[0]), np.square(X[P_est:]-np.roll(X_ml, -1)))
plt.subplot(212)
plt.plot(np.arange(X[P_est:].shape[0]), np.square(X[P_est:]-X_map))

### Bayesian model choice for real audio

In [ ]:
samplerate, grosse = wavfile.read(cwd+'resources/sample audio/grosse_original.wav')
grosse = grosse.astype(np.float64)

In [ ]:
steady_note = grosse[0:int(samplerate*0.351)]

In [ ]:
wavfile.write(cwd+'resources/sample audio/grosse_note.wav', samplerate, steady_note.astype(np.int16))

In [ ]:
steady_note.shape

In [ ]:
def estimate_model_order(data, noise_variance, prior_param_mean, prior_param_variance, P_range, P_step=1):
    # P_range should be a two tuple, also (delta P_range % P_step = 0) for proper operation
    evidences_f = np.zeros(int((P_range[1]-P_range[0])/P_step))
    P_vals = np.arange(P_range[0], P_range[1], step=P_step)
    #print(P_vals)
    for i in range(evidences_f.shape[0]):
        p_i = P_vals[i]
        prior_m = prior_param_mean * np.ones(p_i)
        prior_c_inv = np.linalg.inv(prior_param_variance * np.eye(p_i))
        gi = build_AR_matrix(data, p_i)
        evidences_f[i] = get_log_model_evidence(data, noise_variance, prior_m, prior_c_inv, gi)
    #print(evidences_f)
    return P_vals[np.argmax(evidences_f)]

In [ ]:
def plot_errors(data, P_range, P_step = 1):
    errors_f = np.zeros(int((P_range[1]-P_range[0])/P_step))
    P_vals = np.arange(P_range[0], P_range[1], step=P_step)

    for i in range(errors_f.shape[0]):
        p_i = P_vals[i]
        gi = build_AR_matrix(data, p_i)
        errors_f[i] = np.mean(np.square(data[p_i:] - np.matmul(gi, get_alpha_ml(data, p_i))))

    plt.figure()
    plt.plot(np.arange(start=P_range[0], stop=P_range[1], step=P_step), errors_f)
    plt.xlabel('Model order')
    plt.ylabel('Mean squared residual error')

In [ ]:
plot_errors(steady_note, (1, 101))

In [ ]:
estimate_model_order(steady_note, 4, 0, 1, (200, 310), P_step=10)

In [ ]:
alpha_ml_st = get_alpha_ml(steady_note, 4)

In [ ]:
plot_poles(alpha_ml_st)

In [ ]:
X_ml_st = np.matmul(build_AR_matrix(steady_note, 4), alpha_ml_st)
X_ml_st

In [ ]:
wavfile.write(cwd+'resources/sample audio/steady_note_arproc_300.wav', samplerate, X_ml_st.astype(np.int16))

In [ ]:
np.mean(np.square(X_ml_st - steady_note[4:]))

In [ ]:
estimate_model_order(steady_note, 113, 0, 1, (110, 810), P_step=10)

In [ ]:
alpha_ml_st = get_alpha_ml(steady_note, 250)

In [ ]:
plot_poles(alpha_ml_st)

In [ ]:
X_ml_st = np.matmul(build_AR_matrix(steady_note, 250), alpha_ml_st)
X_ml_st.shape

In [ ]:
wavfile.write(cwd+'resources/sample audio/steady_note_arproc_300.wav', samplerate, X_ml_st.astype(np.int16))

In [ ]:
np.mean(np.square(X_ml_st - steady_note[250:]))

In [ ]:
plt.figure()
plt.subplot(211)
plt.title('Steady note spectrum. Model order: {}'.format(4))
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_st.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(steady_note[4:])))))))
plt.ylabel('True spectrum')
plt.subplot(212)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_st.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_st)))))))
plt.xlabel('Normalised frequency')
plt.ylabel('ML estimate')

### 'Network' steady note

In [ ]:
network_rate, nwst = wavfile.read(cwd+'resources/sample audio/ma_sound.wav')
nwst = nwst.astype(np.float64)
nwst_l = nwst[:,0]
nwst_r = nwst[:,1]

In [ ]:
estimate_model_order(nwst_l, 4, 0, 1, (10, 510), P_step=10)

In [ ]:
alpha_ml_nwst_l = get_alpha_ml(nwst_l, 500)
X_ml_nwst_l = np.matmul(build_AR_matrix(nwst_l, 500), alpha_ml_nwst_l)

In [ ]:
np.mean(np.square(X_ml_nwst_l - nwst_l[500:]))

In [ ]:
estimate_model_order(nwst_l, 355354, 0, 1, (1, 41), P_step=1)

In [ ]:
alpha_ml_nwst_l = get_alpha_ml(nwst_l, 29)
X_ml_nwst_l = np.matmul(build_AR_matrix(nwst_l, 29), alpha_ml_nwst_l)

In [ ]:
np.mean(np.square(X_ml_nwst_l - nwst_l[29:]))

In [ ]:
estimate_model_order(nwst_l, 416522, 0, 1, (1, 41), P_step=1)

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwst_l.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(nwst_l[29:])))))))
plt.subplot(212)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwst_l.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_nwst_l)))))))

In [ ]:
estimate_model_order(nwst_r, 4, 0, 1, (10, 510), P_step=10)

In [ ]:
alpha_ml_nwst_r = get_alpha_ml(nwst_r, 500)
X_ml_nwst_r = np.matmul(build_AR_matrix(nwst_r, 500), alpha_ml_nwst_r)

In [ ]:
np.mean(np.square(X_ml_nwst_r - nwst_r[500:]))

In [ ]:
estimate_model_order(nwst_r, 237190, 0, 1, (20, 70), P_step=1)

In [ ]:
alpha_ml_nwst_r = get_alpha_ml(nwst_r, 48)
X_ml_nwst_r = np.matmul(build_AR_matrix(nwst_r, 48), alpha_ml_nwst_r)

In [ ]:
np.mean(np.square(X_ml_nwst_r - nwst_r[48:]))

In [ ]:
estimate_model_order(nwst_r, 283655, 0, 1, (20, 70), P_step=1)

In [ ]:
alpha_ml_nwst_r = get_alpha_ml(nwst_r, 47)
X_ml_nwst_r = np.matmul(build_AR_matrix(nwst_r, 47), alpha_ml_nwst_r)

In [ ]:
np.mean(np.square(X_ml_nwst_r - nwst_r[47:]))

In [ ]:
estimate_model_order(nwst_r, 283655, 0, 1, (20, 70), P_step=1)

In [ ]:
X_ml_nwst_r.shape, X_ml_nwst_l.shape

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwst_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(nwst_r[47:])))))))
plt.subplot(212)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwst_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_nwst_r)))))))

In [ ]:
X_ml_nwst_r = np.pad(X_ml_nwst_r, (0, X_ml_nwst_l.shape[0]-X_ml_nwst_r.shape[0]))
nwst_ar_proc = np.vstack((X_ml_nwst_l, X_ml_nwst_r)).T
wavfile.write(cwd+'resources/sample audio/nwst_ar_proc.wav', network_rate, nwst_ar_proc.astype(np.int16))

### 'Network' snare transient

In [ ]:
_, nwsnare = wavfile.read(cwd+'resources/sample audio/snare_transient.wav')
nwsnare = nwsnare.astype(np.float64)
nwsnare_l = nwsnare[:,0]
nwsnare_r = nwsnare[:,1]

In [ ]:
estimate_model_order(nwsnare_l, 4, 0, 1, (10, 510), P_step=10)

In [ ]:
alpha_ml_nwsnare_l = get_alpha_ml(nwsnare_l, 500)
X_ml_nwsnare_l = np.matmul(build_AR_matrix(nwsnare_l, 500), alpha_ml_nwsnare_l)

In [ ]:
np.mean(np.square(X_ml_nwsnare_l - nwsnare_l[500:]))

In [ ]:
estimate_model_order(nwsnare_l, 236665, 0, 1, (1700, 1850), P_step=10)

In [ ]:
nwsnare_l.shape

In [ ]:
alpha_ml_nwsnare_l = get_alpha_ml(nwsnare_l, 1760)
X_ml_nwsnare_l = np.matmul(build_AR_matrix(nwsnare_l, 1760), alpha_ml_nwsnare_l)

In [ ]:
np.mean(np.square(X_ml_nwsnare_l - nwsnare_l[1760:]))

In [ ]:
estimate_model_order(nwsnare_l, 61558, 0, 1, (1, 20), P_step=1)

In [ ]:
alpha_ml_nwsnare_l = get_alpha_ml(nwsnare_l, 2568)
X_ml_nwsnare_l = np.matmul(build_AR_matrix(nwsnare_l, 2568), alpha_ml_nwsnare_l)

In [ ]:
np.mean(np.square(X_ml_nwsnare_l - nwsnare_l[2568:]))

In [ ]:
estimate_model_order(nwsnare_l, 26194, 0, 1, (3280, 3292), P_step=1)

In [ ]:
alpha_ml_nwsnare_l = get_alpha_ml(nwsnare_l, 3287)
X_ml_nwsnare_l = np.matmul(build_AR_matrix(nwsnare_l, 3287), alpha_ml_nwsnare_l)

In [ ]:
np.mean(np.square(X_ml_nwsnare_l - nwsnare_l[3287:]))

In [ ]:
estimate_model_order(nwsnare_l, 15275, 0, 1, (3490, 3510), P_step=1)

In [ ]:
alpha_ml_nwsnare_l = get_alpha_ml(nwsnare_l, 3504)
X_ml_nwsnare_l = np.matmul(build_AR_matrix(nwsnare_l, 3504), alpha_ml_nwsnare_l)

In [ ]:
np.mean(np.square(X_ml_nwsnare_l - nwsnare_l[3504:]))

In [ ]:
estimate_model_order(nwsnare_l, 13596, 0, 1, (3499, 3507), P_step=1)

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwsnare_l.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(nwsnare_l[3504:])))))))
plt.subplot(212)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwsnare_l.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_nwsnare_l)))))))

In [ ]:
estimate_model_order(nwsnare_r, 4, 0, 1, (10, 1010), P_step=100)

In [ ]:
alpha_ml_nwsnare_r = get_alpha_ml(nwsnare_r, 910)
X_ml_nwsnare_r = np.matmul(build_AR_matrix(nwsnare_r, 910), alpha_ml_nwsnare_r)

In [ ]:
np.mean(np.square(X_ml_nwsnare_r - nwsnare_r[910:]))

In [ ]:
estimate_model_order(nwsnare_r, 129443, 0, 1, (10, 1010), P_step=100)

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwsnare_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(nwsnare_r[910:])))))))
plt.subplot(212)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwsnare_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_nwsnare_r)))))))

In [ ]:
X_ml_nwsnare_l.shape, X_ml_nwsnare_r.shape

In [ ]:
X_ml_nwsnare_l = np.pad(X_ml_nwsnare_l, (0, X_ml_nwsnare_r.shape[0]-X_ml_nwsnare_l.shape[0]))
nwsnare_ar_proc = np.vstack((X_ml_nwsnare_l, X_ml_nwsnare_r)).T

In [ ]:
wavfile.write(cwd+'resources/sample audio/nwsnare_ar_proc.wav', network_rate, nwsnare_ar_proc.astype(np.int16))

## Simple model order estimation

### Ma sound

In [ ]:
plot_errors(nwst_l, (1, 250))
plot_errors(nwst_r, (1, 250))

In [ ]:
alpha_ml_nwst_r = get_alpha_ml(nwst_r, 8)
X_ml_nwst_r = np.matmul(build_AR_matrix(nwst_r, 8), alpha_ml_nwst_r)
alpha_ml_nwst_l = get_alpha_ml(nwst_l, 8)
X_ml_nwst_l = np.matmul(build_AR_matrix(nwst_l, 8), alpha_ml_nwst_l)

In [ ]:
plt.figure()
plt.subplot(211)
plt.title('Vowel spectrum. Model order: {}'.format(8))
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwst_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(nwst_r[8:])))))))
plt.ylabel('True spectrum')
plt.subplot(212)
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwst_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_nwst_r)))))))
plt.xlabel('Normalised frequency')
plt.ylabel('ML estimate')

In [ ]:
X_ml_nwst_r = np.pad(X_ml_nwst_r, (0, X_ml_nwst_l.shape[0]-X_ml_nwst_r.shape[0]))
nwst_ar_proc = np.vstack((X_ml_nwst_l, X_ml_nwst_r)).T
wavfile.write(cwd+'resources/sample audio/nwma_ar_proc.wav', network_rate, nwst_ar_proc.astype(np.int16))

### Snare transient

In [ ]:
plot_errors(nwsnare_l, (1, 250))
plot_errors(nwsnare_r, (1, 250))

In [ ]:
alpha_ml_nwsnare_r = get_alpha_ml(nwsnare_r, 12)
X_ml_nwsnare_r = np.matmul(build_AR_matrix(nwsnare_r, 12), alpha_ml_nwsnare_r)
alpha_ml_nwsnare_l = get_alpha_ml(nwsnare_l, 12)
X_ml_nwsnare_l = np.matmul(build_AR_matrix(nwsnare_l, 12), alpha_ml_nwsnare_l)

In [ ]:
plt.figure()
plt.subplot(211)
plt.title('Snare transient. Model order: {}'.format(12))
plt.ylabel('True spectrum')
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwsnare_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(nwsnare_r[12:])))))))
plt.subplot(212)
plt.xlabel('Normalised frequency')
plt.ylabel('ML estimate')
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwsnare_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_nwsnare_r)))))))

In [ ]:
X_ml_nwsnare_r = np.pad(X_ml_nwsnare_r, (0, X_ml_nwsnare_l.shape[0]-X_ml_nwsnare_r.shape[0]))
nwsnare_ar_proc = np.vstack((X_ml_nwsnare_l, X_ml_nwsnare_r)).T
wavfile.write(cwd+'resources/sample audio/nwsnare_ar_proc.wav', network_rate, nwsnare_ar_proc.astype(np.int16))

### Th sound

In [ ]:
_, nwth = wavfile.read(cwd+'resources/sample audio/th_sound.wav')
nwth = nwth.astype(np.float64)
nwth_l = nwth[:,0]
nwth_r = nwth[:,1]

In [ ]:
plot_errors(nwth_l, (1, 150))
plot_errors(nwth_r, (1, 150))

In [ ]:
alpha_ml_nwth_r = get_alpha_ml(nwth_r, 10)
X_ml_nwth_r = np.matmul(build_AR_matrix(nwth_r, 10), alpha_ml_nwth_r)
alpha_ml_nwth_l = get_alpha_ml(nwth_l, 10)
X_ml_nwth_l = np.matmul(build_AR_matrix(nwth_l, 10), alpha_ml_nwth_l)

In [ ]:
plt.figure()
plt.subplot(211)
plt.title('Consonant spectrum. Model order: {}'.format(10))
plt.ylabel('True spectrum')
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwth_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(nwth_r[10:])))))))
plt.subplot(212)
plt.xlabel('Normalised frequency')
plt.ylabel('ML estimate')
plt.plot(np.fft.fftshift(np.fft.fftfreq(X_ml_nwth_r.shape[0])), np.log((np.square(np.abs(np.fft.fftshift(np.fft.fft(X_ml_nwth_r)))))))

### Missing packet interpolation

In [ ]:
_, armstrong_missing = wavfile.read(cwd+'resources/Audio for missing packet concealment/armst_37_missing.wav')
armstrong_missing = armstrong_missing.astype(np.float64)

In [ ]:
armstrong_rate, armstrong = wavfile.read(cwd+'resources/sample audio/armst_37_orig.wav')
armstrong = armstrong.astype(np.float64)

In [ ]:
plt.figure()
plt.plot(np.arange(armstrong.shape[0])[:], armstrong[:])
plt.plot(np.arange(armstrong.shape[0])[:], armstrong_missing[:])

In [ ]:
def get_missing_indices_ranges(data):
    # two step process to retrieve all bursts of degradation longer than 2 samples
    # missing samples are zeroed out
    missing_i = np.array([i for i in range(data.shape[0]) if data[i] == 0])
    # to find ranges, we must find the first and last indices of each burst
    beginning_i = np.array([missing_i[i] for i in range(missing_i.shape[0]) if not missing_i[i-1] == missing_i[i]-1])
    ending_i = np.array([missing_i[i] for i in range(-1, missing_i.shape[0]-1) if not missing_i[i+1] == missing_i[i]+1])
    ending_i = np.roll(ending_i, -1)
    # zip to give ranges
    missing_r = np.array(list(zip(beginning_i, ending_i)))
    
    # have to repeat process to fix this method detection single points in the audio
    broken_i = np.empty(0)
    for item in missing_r:
        if (item[1]-item[0])<=1:
            broken_i = np.append(broken_i, item[0])
            broken_i = np.append(broken_i, item[1])

    # rerun the initial process with the singular points in the audio removed
    missing_i = np.array([item for item in missing_i if not (item in broken_i)])
    beginning_i = np.array([missing_i[i] for i in range(missing_i.shape[0]) if not missing_i[i-1] == missing_i[i]-1])
    ending_i = np.array([missing_i[i] for i in range(-1, missing_i.shape[0]-1) if not missing_i[i+1] == missing_i[i]+1])
    ending_i = np.roll(ending_i, -1)
    
    missing_r = np.array(list(zip(beginning_i, ending_i)))
    
    return missing_i, missing_r

In [ ]:
def destroy_packets(data, to_destroy, location):
    temp_data = np.copy(data)
    for num in to_destroy:
            temp_data[int(1000*num+location[0]) : int(1000*num+location[1])] = 0
    return temp_data

In [ ]:
def ar_interpolation(data, alphas_f, alphas_b, missing_i, missing_r, noise_variance):
    prediction_f = np.copy(data) # arrays passed by reference
    # iterate over missing indices
    for index in missing_i:
        term = 0
        # find forward prediction of the process at missing index, checking 
            # if the array bounds are violated
        for i in range(alphas_f.shape[0]): # equivalent to zero initial conditions
            if index-i-1 >=0:
                term += alphas_f[i] * prediction_f[index-i-1]
        # assign forward prediction term along with optional noise
        prediction_f[index] = term + np.sqrt(noise_variance)*np.random.normal(size=1)[0]

    # find reverse prediction by iterating over missing data backwards
    prediction_b = np.copy(data)
    for index in reversed(missing_i):
        term = 0
        for i in range(alphas_b.shape[0]):
            if index+i+1 < data.shape[0]:
                term += alphas_b[i] * prediction_b[index+i+1]
        prediction_b[index] = term + np.sqrt(noise_variance)*np.random.normal(size=1)[0]
    
    # locate each gap in the data and calculate the weighted sum at each point
    prediction_c = np.copy(data)
    for gap in missing_r:
        gap_length = gap[1]-gap[0]
        for i in range(gap_length+1):
            rho = i/(gap_length-1)
            prediction_c[gap[0]+i] = (1-rho)*prediction_f[gap[0]+i] + rho*prediction_b[gap[0]+i]
    
    # return combined prediction
    return prediction_c

In [ ]:
missing_indices, missing_ranges = get_missing_indices_ranges(armstrong_missing)

In [ ]:
plot_errors(armstrong, (1, 101))

In [ ]:
alpha_armstrong_ml = get_alpha_ml(armstrong, 55)
reverse_alpha_armstrong_ml = get_alpha_ml(armstrong[::-1], 55)
#alpha_armstrong_ml = get_alpha_map(armstrong, 27, 177000, 1*np.ones(27), 1*np.eye(27))
#reverse_alpha_armstrong_ml = get_alpha_map(armstrong[::-1], 27, 177000, 1*np.ones(27), 1*np.eye(27))

In [ ]:
def chunk_audio(data, data_rate, chunk_length):
    n = int(chunk_length*data_rate)
    chunks = np.zeros((int(float(data.shape[0])/n)+1, n))

    for i in range(0, data.shape[0], n):
        try:
            chunks[int(i/n), :] = data[i:i+n]
        except ValueError:
            temp_data = np.copy(data[i:i+n])
            temp_data = np.pad(temp_data, (0, n-temp_data.shape[0]))
            chunks[int(i/n), :] = temp_data
    return chunks

def chunk_missing_indices(missing_i, data, chunk_length):
    return np.array([(np.array([i for i in missing_i[j:j+chunk_length]]), j) for j in range(0, data.shape[0], chunk_length)])

In [ ]:
#chunked_armstrong = chunk_audio(armstrong, armstrong_rate, 0.1)
#chunked_armstrong_missing = chunk_audio(armstrong_missing, armstrong_rate, 0.1)

In [ ]:
#chunk_missing_indices(missing_indices, armstrong, int(0.1*armstrong_rate))

In [ ]:
#plt.figure()
#for i in range(armstrong_alphas.shape[1]):
#    plt.hist(armstrong_alphas[:,i], bins = 30, label='Alpha: {}'.format(i+1))
#    plt.legend()

In [ ]:
complete_prediction = ar_interpolation(armstrong_missing, alpha_armstrong_ml, reverse_alpha_armstrong_ml, missing_indices, missing_ranges, 14362)

In [ ]:
np.mean(np.square(complete_prediction - armstrong))

In [ ]:
np.mean(np.square(armstrong_missing-armstrong))

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/armstrong_processed.wav', armstrong_rate, complete_prediction.astype(np.int16))

In [ ]:
plt.figure()
plt.subplot(311)
plt.title('Isolated samples of interpolated audio')
plt.plot(np.arange(armstrong.shape[0])[125-10:158+10], armstrong[125-10:158+10])
plt.plot(np.arange(armstrong.shape[0])[125-10:158+10], complete_prediction[125-10:158+10])
plt.subplot(312)
plt.plot(np.arange(armstrong.shape[0])[3680-10:3708+10], armstrong[3680-10:3708+10])
plt.plot(np.arange(armstrong.shape[0])[3680-10:3708+10], complete_prediction[3680-10:3708+10])
plt.subplot(313)
plt.plot(np.arange(armstrong.shape[0])[10076-10:10104+10], armstrong[10076-10:10104+10])
plt.plot(np.arange(armstrong.shape[0])[10076-10:10104+10], complete_prediction[10076-10:10104+10])
plt.xlabel('Sample number')

### Grosse

In [ ]:
grosserate, grosse_missing = wavfile.read(cwd+'resources/Audio for missing packet concealment/grosse_40_percent_missing.wav')
grosse_missing = grosse_missing.astype(np.float64)

In [ ]:
_, grosse = wavfile.read(cwd+'resources/sample audio/grosse_original.wav')
grosse = grosse.astype(np.float64)

In [ ]:
grosse_missing_i, grosse_missing_r = get_missing_indices_ranges(grosse_missing)

In [ ]:
plot_errors(grosse, (1, 151), P_step=1)

In [ ]:
alpha_grosse_ml = get_alpha_ml(grosse, 77)
reverse_alpha_grosse_ml = get_alpha_ml(grosse[::-1], 77)

In [ ]:
complete_prediction_grosse = ar_interpolation(grosse_missing, alpha_grosse_ml, reverse_alpha_grosse_ml, grosse_missing_i, grosse_missing_r, 0)

In [ ]:
np.mean(np.square(complete_prediction_grosse-grosse))

In [ ]:
np.mean(np.square(grosse_missing-grosse))

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/grosse_processed.wav', grosserate, complete_prediction_grosse.astype(np.int16))

In [ ]:
chunked_grosse = chunk_audio(grosse, grosserate, 0.05)
chunked_grosse.shape

In [ ]:
grosse_alphas = np.zeros((chunked_grosse.shape[0], 20))
for i in range(chunked_grosse.shape[0]):
    grosse_alphas[i,:] = get_alpha_ml(chunked_grosse[i], 20)

In [ ]:
plt.figure()
for i in range(grosse_alphas.shape[1]):
    plt.hist(grosse_alphas[:,i], bins = 15, label='Alpha: {}'.format(i+1))
plt.title('P=20, 50ms Chunks')

### Testing

In [ ]:
armstrong_test = destroy_packets(armstrong, np.linspace(start=0, stop=17, num=18), (200, 210))

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/armstrong_test_1.wav', armstrong_rate, armstrong_test.astype(np.int16))

In [ ]:
missing_i_test, missing_r_test = get_missing_indices_ranges(armstrong_test)

In [ ]:
complete_prediction_test = ar_interpolation(armstrong_test, alpha_armstrong_ml, reverse_alpha_armstrong_ml, missing_i_test, missing_r_test, 0)

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/armstrong_test_processed_1.wav', armstrong_rate, complete_prediction_test.astype(np.int16))

In [ ]:
np.mean(np.square(armstrong-armstrong_test))

In [ ]:
np.mean(np.square(armstrong-complete_prediction_test))

In [ ]:
np.linspace(start=0, stop=18, num=19)

In [ ]:
befores_noise_bad = np.zeros(490)
afters_noise_bad = np.zeros(490)
for i in range(10, 500):
    print(i)
    armstrong_test = destroy_packets(armstrong, np.linspace(start=0, stop=17, num=18), (200, 200+i))
    missing_i_test, missing_r_test = get_missing_indices_ranges(armstrong_test)
    complete_prediction_test = ar_interpolation(armstrong_test, alpha_armstrong_ml, reverse_alpha_armstrong_ml, missing_i_test, missing_r_test, 1000000)
    befores_noise_bad[i-10] = np.mean(np.square(armstrong-armstrong_test))
    afters_noise_bad[i-10] =np.mean(np.square(armstrong-complete_prediction_test))
    
befores_noise_bad, afters_noise_bad

In [ ]:
reductions_noise_bad = (befores_noise_bad-afters_noise_bad)/(befores_noise_bad)

In [ ]:
plt.figure()
plt.plot(np.arange(490)/10, reductions, label='No noise')
plt.plot(np.arange(490)/10, reductions_noise, label='Appropriate noise added')
plt.plot(np.arange(490)/10, abs(reductions_noise_bad), label='Poorly scaled noise added')
plt.xlabel('Proportion of data missing (%)')
plt.ylabel('Reduction in MSE (%)')
plt.legend()

## Challenge 1

In [ ]:
armstrong_challenge = destroy_packets(armstrong, np.array([2]), (0, 600))
wavfile.write(cwd+'resources/Audio for missing packet concealment/armstrong_challenge.wav', armstrong_rate, armstrong_challenge.astype(np.int16))

In [ ]:
# partition data
x_mi_a = armstrong_challenge[:2000]
x_mi_b = armstrong_challenge[2600:]
x_i = armstrong_challenge[2000:2600]

In [ ]:
# initialise A
A = np.zeros((armstrong_challenge.shape[0]-alpha_armstrong_ml.shape[0], armstrong_challenge.shape[0]))
# coefficients vector
a_vec = np.pad(-alpha_armstrong_ml, (0, 1), mode='constant', constant_values=1)
# build A
for i in range(A.shape[0]):
    A[i,:] = np.pad(a_vec, (i, A.shape[1]-i-a_vec.shape[0]))

In [ ]:
# partition A
Ai = np.copy(A[:, 2000:2600])
A_mi_a = np.copy(A[:, :2000])
A_mi_b = np.copy(A[:, 2600:])

In [ ]:
# preliminary calculation
matrix1 = np.matmul(np.linalg.inv(np.matmul(Ai.T, Ai)), Ai.T)

In [ ]:
# build the known data vectors
A_mi = np.hstack((A_mi_a, A_mi_b))
x_mi = np.hstack((x_mi_a, x_mi_b))

In [ ]:
# calculate least squares solution
least_squares = -np.matmul(matrix1, np.matmul(A_mi, x_mi))

In [ ]:
plt.figure()
plt.plot(np.arange(armstrong.shape[0])[1950:2650], armstrong[1950:2650])
plt.plot(np.arange(armstrong.shape[0])[2000:2600], least_squares)

In [ ]:
armstrong_challenge_rep = np.copy(armstrong_challenge)
armstrong_challenge_rep[2000:2600] = least_squares

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/armstrong_challenge_repaired.wav', armstrong_rate, armstrong_challenge_rep.astype(np.int16))

### Applied to supplied audio sample

In [ ]:
# initialise matrices
Ai_final = np.zeros((armstrong_challenge.shape[0]-alpha_armstrong_ml.shape[0], missing_indices.shape[0]))
Ami_final = np.zeros((armstrong_challenge.shape[0]-alpha_armstrong_ml.shape[0], armstrong.shape[0] - missing_indices.shape[0]))
x_mi_final = np.zeros(armstrong.shape[0] - missing_indices.shape[0])

In [ ]:
# build Ai
for i in range(missing_indices.shape[0]):
    Ai_final[:, i] = A[:, missing_indices[i]]

In [ ]:
# build Ami
counter = 0
for i in range(armstrong.shape[0]):
    if i not in missing_indices:
        Ami_final[:, counter] = A[:, i]
        x_mi_final[counter] = armstrong[i]
        counter += 1

In [ ]:
# preliminary calculation
matrix2 = np.matmul(np.linalg.inv(np.matmul(Ai_final.T, Ai_final)), Ai_final.T)

In [ ]:
# least squares solution
least_squares2 = -np.matmul(matrix2, np.matmul(Ami_final, x_mi_final))

In [ ]:
armstrong_least_squares = np.copy(armstrong_missing)

In [ ]:
# apply least squares solution in the correct places
counter2 = 0
for i in range(armstrong.shape[0]):
    if i in missing_indices:
        armstrong_least_squares[i] = least_squares2[counter2]
        counter2+=1

In [ ]:
plt.figure()
plt.plot(np.arange(armstrong.shape[0]), armstrong)
plt.plot(np.arange(armstrong.shape[0]), armstrong_least_squares)

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/armstrong_least_squares.wav', armstrong_rate, armstrong_least_squares.astype(np.int16))

In [ ]:
np.mean(np.square(armstrong - armstrong_least_squares))

In [ ]:
np.mean(np.square(armstrong - complete_prediction))

In [ ]:
def least_squares_interpolator(data, alphas):
    missing_i, _ = get_missing_indices_ranges(data)
    # initialise matrices
    A_f = np.zeros((data.shape[0]-alphas.shape[0], data.shape[0]))
    Ai_f = np.zeros((data.shape[0]-alphas.shape[0], missing_i.shape[0]))
    Ami_f = np.zeros((data.shape[0]-alphas.shape[0], data.shape[0] - missing_i.shape[0]))
    x_mi_f = np.zeros(data.shape[0] - missing_i.shape[0])

    # coefficients vector
    a_vec_f = np.pad(-alphas, (0, 1), mode='constant', constant_values=1)
    # build A
    for i in range(A_f.shape[0]):
        A_f[i,:] = np.pad(a_vec_f, (i, A_f.shape[1]-i-a_vec_f.shape[0]))
    # build Ai
    for i in range(missing_i.shape[0]):
        Ai_f[:, i] = A_f[:, missing_i[i]]  
    # build Ami
    c = 0
    for i in range(data.shape[0]):
        if i not in missing_i:
            Ami_f[:, c] = A_f[:, i]
            x_mi_f[c] = data[i]
            c += 1
    # preliminary calculation
    mat = np.matmul(np.linalg.inv(np.matmul(Ai_f.T, Ai_f)), Ai_f.T)
    ls = -np.matmul(mat, np.matmul(Ami_f, x_mi_f))
    # apply least squares solution in the correct places
    c2 = 0
    ls_data = np.copy(data)
    for i in range(data.shape[0]):
        if i in missing_i:
            ls_data[i] = ls[c2]
            c2+=1
    return ls_data

In [ ]:
function_test = np.copy(armstrong_challenge)

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot(np.arange(armstrong.shape[0])[:5000], armstrong[:5000])
plt.plot(np.arange(armstrong.shape[0])[:5000], function_test_rep[:5000])
plt.subplot(212)
plt.plot(np.arange(armstrong.shape[0])[2000:2600], armstrong[2000:2600])
plt.plot(np.arange(armstrong.shape[0])[2000:2600], function_test_rep[2000:2600])
plt.xlabel('Sample')

In [ ]:
function_test_rep = least_squares_interpolator(function_test, alpha_armstrong_ml)

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/function_test.wav', armstrong_rate, function_test_rep.astype(np.int16))

In [ ]:
np.mean(np.square(armstrong - function_test_rep)), np.mean(np.square(armstrong - function_test))

In [ ]:
wavfile.write(cwd+'resources/Audio for missing packet concealment/grosse_least_squares.wav', grosserate, least_squares_interpolator(grosse_missing, alpha_grosse_ml).astype(np.int16))